In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d #pre written interpolation function

In [64]:
BETA = .9
GAMMA = .5772 #euler's constant
THETA = 1.1
N = 7
BIDS = np.arange(1,16)
STATES = np.arange(1,11)

In [65]:
def cost(states, shock):
    return THETA*states + shock


def profit(contract, bids, states, shock):
    return contract*( bids< np.min(bids) )*(bids - cost(states,shock))  


def backlog(contract, bids, states):
    return contract*(  bids< np.min(bids) ) + states


print cost(np.array([1,2]),np.array([2,1]))
print profit(1,np.array([4,5]),np.array([0,1]),np.array([1,2]))
print backlog(1,np.array([4,5]),np.array([0,1]))

[3.1 3.2]
[0. 0.]
[0 1]


In [67]:
def cost(states, shock):
    """ states are all states, the shock is just an integer"""
    return THETA*states + shock


def profit(contract, my_bid, other_bids, state, shock):
    """this returns the same dimension as my bid
    
    will need to tile bids if evaluating multiple at once"""
    return contract*( my_bid < np.min(other_bids) )*( my_bid - cost(state,shock) )  


def backlog(contract, my_bid, other_bids, state):
    """this returns the same dimension as my bid
    
     will need to tile bids if evaluating multiple at once"""
    return np.minimum( np.max(STATES), contract*(  my_bid < np.min(other_bids) ) + state )


print cost(np.array([1,2]),np.array([2,1]))
print profit(1,2,np.array([4,5]),1,1)
print backlog(1,2,np.array([4,5]),1)

[3.1 3.2]
-0.10000000000000009
2


In [90]:
def operator(initial_values, other_bids):
    """Calculate the value of the 
    Bellman operator in the value function
    
    update the value function and the policy function
    for player i, given the other players value and policy
    functions"""
    
    #initialize contract and state to 0
    contract = 1
    shock = 0
    
    #possible states for just one player
    states = STATES.reshape((1,len(STATES),1))
    states = np.tile(states, (len(BIDS),1,N))
    
    #possible bids for all states, again 1 player
    bids = BIDS.reshape((len(BIDS), 1, 1))
    bids = np.tile(bids,(1,len(STATES),N))
    
    #interpolate for easy access to value function
    value = interp1d(STATES,initial_values)
    
    #calculate value of each bid and take max in each state
    v_next = profit(contract,bids,other_bids,states,shock)
    v_next = v_next + BETA*value(backlog(contract, bids, other_bids, states))
    
    #find maximum and argmax
    policy = np.argmax(v_next,axis=0)
    policy = BIDS[policy]
    v_next = np.max(v_next,axis=0)
    
    return policy, v_next



#initialize policy as something else
foo_policy = lambda state: 5
foo_bids = [foo_policy]*(N-1)

v0 = np.concatenate((np.zeros(5),2*np.ones(5)))
operator(v0, 6 + np.ones(N-1) )

(array([[6, 6, 6, 6, 6, 6, 6],
        [6, 6, 6, 6, 6, 6, 6],
        [6, 6, 6, 6, 6, 6, 6],
        [6, 6, 6, 6, 6, 6, 6],
        [6, 6, 6, 6, 6, 6, 6],
        [7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7],
        [7, 7, 7, 7, 7, 7, 7]]), array([[4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9],
        [3.8, 3.8, 3.8, 3.8, 3.8, 3.8, 3.8],
        [2.7, 2.7, 2.7, 2.7, 2.7, 2.7, 2.7],
        [1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6],
        [2.3, 2.3, 2.3, 2.3, 2.3, 2.3, 2.3],
        [1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8],
        [1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8],
        [1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8],
        [1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8],
        [1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8]]))

array([2, 3])

In [ ]:
def nash_equilibrium(value_function):
    """given the value function 
    find the symmetric nash equilibrium
    in each state"""

In [ ]:
def  value_function(initial_value, error, maxiter):
    """calculate the value function in a symmetric
        markov perfect equilibria"""
    
    #initialize loop variables
    v0 = np.ones(len(states))
    v_next = operator(a, theta1, cost, v0, v1)
    
    while  maxiter >= 0 and ( 
            np.abs(v0 - v0_next ).max() > error
           or np.abs(v1 - v1_next ).max()  > error ):
        
        #iterate loop variables for each player
        for i in range(N):
            v0, v1 = v0_next, v1_next
            v0_next, v1_next = operator(a, theta1, cost, v0, v1)
            maxiter = maxiter -1

    return  value